In [391]:
import pandas as pd
import numpy as np
import os

In [392]:
df = pd.read_csv('./concat_inputs_preTag_v2_24-03-12.csv', index_col=0)
df = df[['VH_AA', 'VL_AA', 'antigen_seq', 'antigen_name','source']]
df.shape

C:\Users\perry\AppData\Local\Temp\ipykernel_35436\2235230924.py:1: DtypeWarning: Columns (6,7,9,11,12,13,15,17,21,22,23,24,25,26,27,28,29,30,31,32,33,34,40,41,42,43,44,45,47,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./concat_inputs_preTag_v2_24-03-12.csv', index_col=0)


(18648, 5)

In [393]:
df['antigen_seq'].nunique()

1469

In [394]:
df = df[df['source'] != 'SAbDab']
df['antigen_seq'].nunique()

70

In [395]:
antigen_df = df[['antigen_name', 'antigen_seq']].drop_duplicates(subset='antigen_seq')

### SignalP

In [396]:
antigen_df.index = antigen_df.index + ':' + antigen_df['antigen_name']
antigen_df.head()

,antigen_name,antigen_seq
007_11_4A04:H1N1,H1N1,MKAILVVLLYTFATANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...
ADI-15731:EBOVZaire sGP,EBOVZaire sGP,MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSIPLGVIHNSTLQVS...
ADI-15732:EBOVZaire GP∆TM,EBOVZaire GP∆TM,MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSIPLGVIHNSTLQVS...
1198_01_A06:HCV,HCV,RNASGVYHVTNDCPNSSIVYEAEDLILHAPGCVPCVRQGNVSRCWV...
01.a.01_Heavy:H5N1,H5N1,MEKIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...


In [397]:
# Writing fasta for SignalP
# with open('nonSAbDab_antigens_for_SignalP_24-03-12.fasta', 'w') as file:
#     for i,row in antigen_df.iterrows():
#         file.write('>' + row['antigen_name'] + '\n')
#         file.write(row['antigen_seq'] + '\n')

# file.close()

Used slow version with short output, for 'Other' species

In [398]:
### SignalP output

signalP_output = pd.read_csv('./signalP/' + 'signalP_summary_output_24-03-12.txt', sep='\t', header=1, index_col=0)
signalP_output['antigen_name'] = signalP_output.index
signalP_output.index = antigen_df.index
signalP_output = signalP_output[signalP_output['Prediction'] != 'OTHER']
print(signalP_output.shape)

signalP_output['Pos'] = signalP_output['CS Position'].apply(lambda x: x.split('CS pos: ')[1].split('.')[0])
signalP_output['Prob'] = signalP_output['CS Position'].apply(lambda x: float(x.split('Pr: ')[1]))

# Only keep high probability
signalP_output = signalP_output[signalP_output['Prob'] >= 0.8]
print(signalP_output.shape)

(41, 9)
(39, 11)


In [399]:
def get_signalP(row):

    antigen_seq = row['full_antigen']
    index = row.name
    cleave_pos = int(row['Pos'].split('-')[1])

    signal_peptide = antigen_seq[0:cleave_pos]

    return signal_peptide

In [400]:
signalP_output['full_antigen'] = antigen_df['antigen_seq']

In [401]:
signalP_output['signal_peptide'] = signalP_output.apply(get_signalP, axis=1)

In [402]:
signalP_output['signal_peptide'].value_counts()

MFVFLVLLPLVSSQ                       13
MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSI     4
MSWKVVIIFSLLITPQHGL                   3
MEKIVLLLAIVSLVKSD                     2
MKAILVVLLYTFATANAD                    1
MNTQILVFALIAIIPTNAD                   1
MGGLSLLQLPRDKFRKSSFFVWVIILFQKAFSM     1
MELGLSWIFLLAILKGVQCQ                  1
ATMELLILKANAITTILTAVTFCFASGQ          1
MELLIHRSSAIFLTLAINALYLTSSQ            1
MYKIVVIIALLGAVKGL                     1
MKTIIALSYILCLVFAQ                     1
MAIIYLILLFTAVRGD                      1
MPMGSLQPLATLYLLGMLVASVLAA             1
MKAKLLVLLCTFTATYAD                    1
MFLILLISLPTAFAV                       1
MFIFLLFLTLTSGS                        1
MKAILVVLLYTFTTANAD                    1
MELLIHRSSAIFLTLAVNALYLTSSQ            1
MELLILKANAITTILTAVTFCFASGQ            1
MFVFLFVLPLVSSQ                        1
Name: signal_peptide, dtype: int64

In [403]:
signalP_output['clean_antigen'] = signalP_output.apply(lambda x: x['full_antigen'].replace(x['signal_peptide'], ''), axis=1)

In [404]:
cleaned_dict = dict(zip(signalP_output['full_antigen'], signalP_output['clean_antigen']))

## Mapping cleaned antigens into SAbDAb df

df['antigen_noSignalP'] = df['antigen_seq'].replace(cleaned_dict)

assert df['antigen_noSignalP'].isna().sum() == 0

### Detagging

In [405]:
df[df['antigen_noSignalP'].apply(lambda x: x.count('X') > 2)]

,VH_AA,VL_AA,antigen_seq,antigen_name,source,antigen_noSignalP


In [406]:
df[df['antigen_noSignalP'].apply(lambda x: x.count('|') > 2)]

,VH_AA,VL_AA,antigen_seq,antigen_name,source,antigen_noSignalP


In [407]:
len(X_ant)

16400

In [408]:
X_ant = df['antigen_noSignalP'].copy()

In [409]:
his_tag = 'GSHHHHHHGS'
his_tag6 ='HHHHHHH'
strep_tag = 'WSHQPFEK'
strep_tagII = 'WSHPQFEK'
strep_tagIII = 'SAWSHPQFEK'
flexible_connector = 'GGGSGGGGSGGS'
twin_strep = 'SAWSHPQFEKGGGSGGGGSGGSAWSHPQFEK'
thrombin = 'LVPRGS'
full_thrombin = 'MGSSHHHHHHSSGLVPRGSHM'

spytag001 = 'AHIVMVDAYKPTK'
spytag003 = 'RGVPHIVMVDAYKRYK'

myc_tag = 'EQKLISEEDL'
flagtag = 'DYKDDDDK'

# https://www.uniprot.org/uniprotkb/O55244/entry#sequences
cloning_site = 'MGSSHHHHHHSSGLVPRGSHMISNSSLVPLALVSSPVSSGRGSHHHHHHHHHH'

hrv3c = 'LEVLFQGP'
avi_tag = 'GLNDIFEAQKIEWHE'

tags = [his_tag, his_tag6, strep_tag, strep_tagII, strep_tagIII, flexible_connector, avi_tag, hrv3c, twin_strep, thrombin, 
        spytag001, spytag003, myc_tag, flagtag, full_thrombin]

In [410]:
tagged_dfs = {}
for tag in tags:
    tag_df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(tag_df) > 0:

        tagged_dfs[tag] = tag_df

In [411]:
tagged_dfs.keys()

dict_keys(['GLNDIFEAQKIEWHE', 'LEVLFQGP', 'YPYDVPDYA'])

In [412]:
def find_and_replace_tag(seq, tag):
    seq_len = len(seq)
    split_seq = seq.split(tag)

    if len(split_seq) == 2:

        if len(split_seq[0]) < 20:
            print(split_seq[0])
            return split_seq[1]
        
        if len(split_seq[1]) < 20:
            print(split_seq[1])
            return split_seq[0]
        
        else:
            print('long attachment:')
            print(seq)
            print('---')
            
            return seq
        
    if len(split_seq) >2:
        print('>2 splits')
        return seq
    
    if len(split_seq) == 1:
        return seq
    

In [413]:
# Removing his tags from beginning of these
# X_ant.loc[tagged_dfs[his_tag].index] = tagged_dfs[his_tag].apply(lambda x: x.split(his_tag)[1])

# X_ant.loc[tagged_dfs[his_tag6].index] = tagged_dfs[his_tag6].apply(lambda x: x.split(his_tag6)[1])

# X_ant = X_ant.apply(lambda x: fine_and_replace_tag(x, his_tag))
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'HHHHHH'))
X_ant = test[test != True]

A lot of these have long tags. Will try removing histines again at the end.

In [414]:
tagged_dfs = {}
for tag in tags:
    tag_df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(tag_df) > 0:
        tagged_dfs[tag] = tag_df

tagged_dfs.keys()

dict_keys(['GLNDIFEAQKIEWHE', 'LEVLFQGP', 'YPYDVPDYA'])

In [415]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'GLNDIFEAQKIEWHE'))
X_ant = test[test != True]

G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G















































































































G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G
G


In [416]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'LEVLFQGP'))
X_ant = test[test != True]

In [430]:
X_ant

007_11_4A04                                         TLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGV...
007_11_4B04                                         TLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGV...
007_11_4B06                                         TLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGV...
007_11_4E01                                         TLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGV...
007_11_4E05                                         TLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGV...
                                                                          ...                        
covabdab:SARS-CoV2_WT:S; Unk:368.02a.C.0191:4509    CVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS...
covabdab:SARS-CoV2_WT:S; Unk:368.01a.A.0022:4238    CVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS...
covabdab:SARS-CoV2_WT:S; Unk:368.01a.A.0037:4251    CVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS...
covabdab:SARS-CoV2_WT:S; Unk:368.07.C.0004:4889     CVNLTTRTQLPPAYTNSFTRGVYYPDKVFR

In [431]:
df['detagged_ant'] = X_ant

In [432]:
df.to_csv('nonSabdab_detagged_24-03-12.csv')